In [1]:
%pip install cohere tiktoken

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

In [3]:
import os
import sys
import pkg_resources
print("Python interpreter:", sys.executable)
print("Packages:")
for dist in pkg_resources.working_set:
    print(dist)


Python interpreter: /home/meimfhd/fhd/langchain-apis/.venv/bin/python
Packages:
regex 2023.12.25
fsspec 2023.12.2
certifi 2023.11.17
setuptools 59.6.0
kubernetes 28.1.0
pyzmq 25.1.2
flatbuffers 23.5.26
attrs 23.2.0
packaging 23.2
pip 22.0.2
coloredlogs 15.0.1
websockets 12.0
humanfriendly 10.0
ipython 8.19.0
jupyter-client 8.6.0
tenacity 8.2.3
click 8.1.7
overrides 7.4.0
ipykernel 6.28.0
importlib-metadata 6.11.0
tornado 6.4
importlib-resources 6.1.1
multidict 6.0.4
PyYAML 6.0.1
traitlets 5.14.1
psutil 5.9.7
jupyter-core 5.7.0
cachetools 5.3.2
decorator 5.1.1
tqdm 4.66.1
cohere 4.40
protobuf 4.25.1
beautifulsoup4 4.12.2
pexpect 4.9.0
rsa 4.9
typing-extensions 4.9.0
anyio 4.2.0
bcrypt 4.1.2
platformdirs 4.1.0
async-timeout 4.0.3
mmh3 4.0.1
marshmallow 3.20.1
zipp 3.17.0
filelock 3.13.1
aiohttp 3.9.1
asgiref 3.7.2
idna 3.6
pulsar-client 3.4.0
charset-normalizer 3.3.2
oauthlib 3.2.2
posthog 3.1.0
prompt-toolkit 3.0.43
greenlet 3.0.3
types-requests 2.31.0.6
requests 2.31.0
google-auth 2.26

In [4]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [5]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    )
)

In [6]:
docs = loader.load()

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [8]:
splits = text_splitter.split_documents(docs)

In [11]:

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [12]:

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [13]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [14]:
rag_chain.invoke("What is Task Decomposition?")

"Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. It allows for better planning and understanding of the task at hand. This approach, such as the Chain of Thought (CoT) and Tree of Thoughts, helps enhance model performance and provides insights into the model's thinking process."

In [15]:
# cleanup
vectorstore.delete_collection()